# TP Métaheuristiques

## 1ère étape

### Lecture des jeux de données

Remarque : Les arcs enregistrés dans les fichiers ne sont pas forcément dans le bon sens.
Exemple : 4 2 <=> 2 4

In [22]:
# Objets
class Arbre:
    # Liste des noeuds à évacuer
    # Nombre de noeuds à évacuer
    # Le noeud sur
    
    def __init__(self):
        self.listeNoeudsE = []
    def sur(self, c):
        self.nombreNoeudsE = c[0]
        self.noeudSur = c[1]
    def ajoutNoeudE(self, c):
        nouveauNoeudE = NoeudAEvacuer(c)
        self.listeNoeudsE.append(nouveauNoeudE)
        
class NoeudAEvacuer:
    # ID/Population/taux d'évacuation max
    # Chemin d'évacuation
    def __init__(self, c):
        self.nom = c[0]
        self.pop = c[1]
        self.tauxE = c[2]
        self.cheminE = c[3:]

class Arc: 
    # Arc origine/destination(int) ??  /Durée(int)/Capa(int) 
    def __init__(self, c):
        self.origine = c[0]
        self.destination = c[1]
        self.duree = c[3]
        self.capa = c[4]
        
class Graphe: 
    # Liste d'arcs UTILES ??
    # ??????????? Liste noeuds UTILES ??
    # Nombre d'arcs UTILES ??
    # Nombre de noeuds UTILES ??
    def __init__(self):
        self.listeArcs = []
    def nombre(self,c):
        self.nombreNoeuds = c[0]
        self.nombreArcs = c[1]
    def ajoutArc(self, c):
        nouveauArc = Arc(c)
        self.listeArcs.append(nouveauArc)
    """
    def majNombreArc(self):
        for arc in self.listeArcs:
            self.nombreArc = self.nombreArcs + 1
    def majNombreNoeuds(self):
        for noeud in self.listeNoeuds:
            self.nombreNoeuds = self.nombreNoeuds + 1
    """ 
     
# -------------------------------

def lecture(nomFichier): 
    with open("./Instances/" + nomFichier, "r") as fichier :
        print ("Name of the file: " + fichier.name)

        # Liste des lignes du fichier
        lignes = fichier.readlines()
        
        # Traitement de l'instance
        val = 1
        for ligne in lignes[1:]: 
            c = ligne.split()
            if c[0] == 'c': 
                val = 2
            else:
                if val == 1:
                    traitementArbre(c)
                elif val == 2:
                    traitementGraphe(c)
                else :
                    print("Erreur")
                
        #graphe.majNombreArcs()
        #graphe.majNombreNoeuds()
    
    
    
        fichier.close()

# ---------------------------------

# Fonctions
def traitementArbre(c):
    if len(c) != 2:
        arbre.ajoutNoeudE(c)
    else:
        arbre.sur(c)
        
def traitementGraphe(c):
    if len(c) != 2: 
        graphe.ajoutArc(c)
    else:
        graphe.nombre(c)
        
def affichage():
    print("=============================================")
    print("--- A R B R E ---")
    print("Noeud sûr : %s." %(arbre.noeudSur))
    print("Nombre de noeuds à évacuer : %s." %(arbre.nombreNoeudsE)) 
    print("ID - Pop - TauxE (+ Chemin)")
    for noeudE in arbre.listeNoeudsE:
        print("%s     %s     %s" %(noeudE.nom, noeudE.pop, noeudE.tauxE))
        print(*noeudE.cheminE, sep = " ==> ")
        print("------------------------------------")
    print("--- G R A P H E ---")
    print("Nombre de noeuds : %s." %(graphe.nombreNoeuds))
    print("Nombre d'arcs : %s." %(graphe.nombreArcs))
    print("Orig/Dest -- Durée -- Capacité")
    for arc in graphe.listeArcs:
        print("  %s/%s       %s      %s" %(arc.origine, arc.destination, arc.duree, arc.capa))
        print("------------------------------------")
    print("=============================================")
    

In [23]:
arbre = Arbre()
graphe = Graphe()

lecture("exemple_cours.txt")
affichage()

Name of the file: ./Instances/exemple_cours.txt
--- A R B R E ---
Noeud sûr : 6.
Nombre de noeuds à évacuer : 3.
ID - Pop - TauxE (+ Chemin)
1     48     8
3 ==> 4 ==> 5 ==> 6
------------------------------------
2     30     5
3 ==> 4 ==> 5 ==> 6
------------------------------------
3     33     3
2 ==> 5 ==> 6
------------------------------------
--- G R A P H E ---
Nombre de noeuds : 7.
Nombre d'arcs : 8.
Orig/Dest -- Durée -- Capacité
  0/1       100.45      50.2
------------------------------------
  0/2       100.45      50.2
------------------------------------
  0/3       100.45      50.2
------------------------------------
  1/4       7      8
------------------------------------
  2/4       4      5
------------------------------------
  3/5       6      3
------------------------------------
  4/5       9      10
------------------------------------
  5/6       12      11
------------------------------------


Je pense qu'on peut enlever les arcs inutiles, en vérifiant si ils sont dans un des chemins d'évacuation des noeudsE.
    --> Penser alors à modifier la valeur de graphe.nombreArc ..